Here a code for crawler web

Can craw only 1 page

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time

url = "https://naya.moph.go.th/table.php?p=databaseicd10"

# Initialize WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in headless mode (optional)
service = Service(verbose = True)
driver = webdriver.Chrome(service=service, options=options)

try:
    # Open the target URL
    driver.get(url)
    time.sleep(5)  # Wait for content to load (adjust if needed)

    # Locate the table
    table = driver.find_element(By.TAG_NAME, "table")
    rows = table.find_elements(By.TAG_NAME, "tr")

    # Extract data
    data = []
    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        data.append([col.text for col in cols])

    # Convert to DataFrame
    headers = ["รหัสโรค", "ชื่อสามัญ", "ชื่อไทย"]  # Adjust headers as necessary
    df = pd.DataFrame(data[1:], columns=headers)  # Skip the header row in data

    # Save to CSV
    output_file = "icd10_data.csv"
    df.to_csv(output_file, index=False, encoding="utf-8")
    print(f"Data saved to {output_file}")

finally:
    # Close the WebDriver
    driver.quit()


Data saved to icd10_data.csv


Try to craw all page

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time

url = "https://naya.moph.go.th/table.php?p=databaseicd10"

# Initialize WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in headless mode (optional)
service = Service(verbose = True)
driver = webdriver.Chrome(service=service, options=options)

try:
    # Open the target URL
    driver.get(url)
    time.sleep(5)  # Wait for content to load (adjust if needed)

    # Initialize list to store all scraped data
    all_data = []
    
    while True:
        # Locate the table
        table = driver.find_element(By.TAG_NAME, "table")
        rows = table.find_elements(By.TAG_NAME, "tr")

        # Extract data from the table rows
        data = []
        for row in rows:
            cols = row.find_elements(By.TAG_NAME, "td")
            data.append([col.text for col in cols])

        # Append the data of the current page (skip header)
        all_data.extend(data[1:])  # Skipping the header row

        # Check if "Next" button is present (for pagination)
        try:
            next_button = driver.find_element(By.XPATH, '//a[@aria-controls="dataTable" and contains(text(), "ถัดไป")]')
            next_button.click()
            time.sleep(3)  # Wait for the next page to load (adjust if needed)
        except:
            # No more pages, break the loop
            print("Reached the last page.")
            break

    # Convert to DataFrame
    headers = ["รหัสโรค", "ชื่อสามัญ", "ชื่อไทย"]  # Adjust headers as necessary
    df = pd.DataFrame(all_data, columns=headers)

    # Save to CSV
    output_file = "icd10_data.csv"
    df.to_csv(output_file, index=False, encoding="utf-8")
    print(f"Data saved to {output_file}")

finally:
    # Close the WebDriver
    driver.quit()
